In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colormaps
from scipy.signal import find_peaks, convolve, medfilt
from scipy.optimize import curve_fit
import matplotlib as mpl
from glob import glob
from copy import copy

%matplotlib widget

In [ ]:
freqs, s21 = np.load('./VNAsweep.npy')

In [ ]:
sw = 4 # even number
mph = 0.05
window = np.ones(sw)/sw
smooth_s21 = convolve(s21, window, mode='same')
ds21 = np.diff(smooth_s21, 1)
smooth_ds21 = convolve(ds21, window, mode='same')
d2s21 = np.diff(smooth_ds21, 1)

fig, ax = plt.subplot_mosaic('a;b', sharex=True, figsize=(12, 6), constrained_layout=True)
# ax['a'].plot(smooth_ds21)
ax['a'].plot(freqs[:-2], d2s21, lw=.5)
ax['b'].plot(freqs, s21, lw=.5)
ax['a'].axhline(mph, c='r')
# ax['b'].plot(s21)

locs, props = find_peaks(d2s21, height=mph, prominence=mph)
heights = props['peak_heights']
ax['a'].scatter(freqs[:-2][locs], heights, marker='v', c='None', edgecolor='g')
# ax['a'].scatter(freqs[locs[(locs[1:]-locs[:-1]) < 10]], heights, marker='v', c='None', edgecolor='r')
ax['b'].scatter(freqs[locs], s21[locs], marker='^', c='None', edgecolor='g')
ax['b'].set_xlabel('Frequency [GHz]')
print(len(locs))

In [ ]:
dir = './S21s/'
dark_s21 = np.load(glob(dir + '*dark*.npy')[0])
freqs = np.load(glob(dir + '*freqs*.npy')[0])
sw = 4 # even number
mph = 0.05
window = np.ones(sw)/sw
smooth_s21 = convolve(dark_s21, window, mode='same')
ds21 = np.diff(smooth_s21, 1)
smooth_ds21 = convolve(ds21, window, mode='same')
d2s21 = np.diff(smooth_ds21, 1)

fig, ax = plt.subplot_mosaic('a;b', sharex=True, figsize=(12, 6), constrained_layout=True)
# ax['a'].plot(smooth_ds21)
ax['a'].plot(freqs[:-2], d2s21, lw=.5)
ax['b'].plot(freqs, dark_s21, lw=.5)
ax['a'].axhline(mph, c='r')
# ax['b'].plot(s21)

locs, props = find_peaks(d2s21, height=mph, prominence=mph)
heights = props['peak_heights']
ax['a'].scatter(freqs[:-2][locs], heights, marker='v', c='None', edgecolor='g')
# ax['a'].scatter(freqs[locs[(locs[1:]-locs[:-1]) < 10]], heights, marker='v', c='None', edgecolor='r')
ax['b'].scatter(freqs[locs], dark_s21[locs], marker='^', c='None', edgecolor='g')
ax['b'].set_xlabel('Frequency [GHz]')
print(len(locs))

In [3]:
yfiles = glob(dir + '*x0*.npy')
nr_y = len(yfiles)
nr_steps = nr_y
nr_kids = len(locs)
s21s = np.empty((nr_steps, nr_kids))
s21s[:] = np.nan
s0 = dark_s21[locs]
sy0 = np.load(yfiles[0])[locs]
for i, yfile in enumerate(yfiles):
    if i < len(yfiles):
        yfile = yfiles[i]
        s21y = np.load(yfile)
        s21s[i, :]  = s21y[locs] - sy0 

In [ ]:
sw = 2
dw = 4
medw = 3
mph = .25
mpw = 0
mpp = mph/2
smooth_s21s = np.empty((len(yfiles)+1, len(locs)))
smooth_s21s[:] = np.nan
smooth_d2s21s = np.empty((len(yfiles)+6, len(locs)))
smooth_d2s21s[:] = np.nan
window = np.ones(sw)/sw
dwindow = np.ones(dw)/dw
pos = np.empty((nr_kids, 3))
pos[:] = np.nan
pos[:, 2] = np.arange(nr_kids)
semi_pos = np.empty((nr_kids, 3))
semi_pos[:] = np.nan
semi_pos[:, 2] = np.arange(nr_kids)
nr_good = 0
nr_semi = 0
bad = []
for i in range(nr_kids):
    powery = s21s[:, i]
    powery = powery[~np.isnan(powery)]
    medpowery = medfilt(powery, medw)
    smoothpowery = convolve(medpowery, window, mode='full')
    smooth_s21s[:len(smoothpowery), i] = smoothpowery
    d1y = np.diff(smoothpowery, 1)
    d1ysmooth = convolve(d1y, dwindow, mode='full')
    d2y = np.diff(d1ysmooth, 1)
    d2y = convolve(d2y, dwindow, mode='full')
    smooth_d2s21s[:len(d2y), i] = d2y
    locy, propy = find_peaks(-d2y, height=mph, prominence=mpp, width=mpw)
    
    if len(locy)==1:
        cy = np.round((propy['right_ips'] + propy['left_ips']) / 2)
        pos[i, 1] = cy
        nr_good += 1
    else:
        bad.append(i)
        pass
print('good: ', nr_good)
print('bad: ', len(bad))

In [ ]:
kid = 35
y = s21s[:, kid]
ysmooth = smooth_s21s[:, kid]
d2ysmooth = smooth_d2s21s[:, kid]

fig, ax = plt.subplot_mosaic('b;d;f', figsize=(4, 9), sharex=True)
ax['b'].plot(y)

ax['d'].plot(ysmooth)
ax['d'].scatter(np.nanargmax(ysmooth), np.nanmax(ysmooth), c='r')

ax['f'].plot(d2ysmooth)
ax['f'].axhline(-mph, c='r')
locy, propy = find_peaks(-d2ysmooth, height=mph, prominence=mpp, width=mpw)
cy = np.round((propy['right_ips'] + propy['left_ips']) / 2)
ax['f'].scatter(locy, -propy['peak_heights'], c='r')
ax['f'].set_ylim(-1, 1)
ax['f'].scatter(cy, -propy['peak_heights'], c='None', edgecolor='g')

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))
full_pos = np.empty((nr_kids, 3))
full_pos[:] = np.nan
full_pos[:, 2] = np.arange(nr_kids)
ax.scatter(*pos[:, :2].T)
ax.scatter(*semi_pos[:, :2].T)
gridx = np.array([5, 8, 12, 20, 25, 30])
gridy = np.array([3, 7.5, 10.5, 15, 24, 28])
for i in range(len(gridx)):
    ax.axvline(gridx[i], c='k', lw=.5)
    ax.axhline(gridy[i], c='k', lw=.5)
for coord in pos:
    if ~np.any(np.isnan(coord)):
        text = ax.text(coord[0]+.5, coord[1]+.5, str(int(coord[2])), ha="center", va="center", color="k", fontsize='small',zorder=4)  
for coord in semi_pos:
    if ~np.any(np.isnan(coord)):
        text = ax.text(coord[0]+.5, coord[1]+.5, str(int(coord[2])), ha="center", va="center", color="k", fontsize='small',zorder=4)  
full_pos[:, :2] = pos[:, :2] + semi_pos[:, :2]

In [ ]:
kid_id_board = np.load('KID_ID_BOARD_6x6.npy')
kid_id_board -= 1
# kid_id_board = np.flip(kid_id_board, axis=0)
kid_id_board

In [ ]:
kids = np.array([[0, 31, 8, 17, 2, 32],
                 [9, 18, 1, 33, 25, 15], 
                 [30, 7, 24, 16, 34, 3],
                 [19, 10, 35, 27, 23, 14],
                 [6, 29, 12, 22, 4, 26],
                 [11, 20, 5, 28, 13, 21]
                  ], dtype=float)
kids[:, -1] = np.nan
kids[0, :] = np.nan
kids[-1, :] = np.nan
kids

In [19]:
M_pxs = 6
N_pxs = 6
mask = (kids == kid_id_board)
nr_tot = (M_pxs*N_pxs)
nr_detected = nr_kids
nr_notswapped = np.sum(mask)
nr_identified = np.sum(~np.isnan(kids))
nr_swapped = nr_identified - nr_notswapped
same = np.argwhere(mask)
ids = np.rint(np.linspace(0, 399, M_pxs*N_pxs)).astype(int)
# f0s_design = np.load('LEKID_EBEAM_f0s_5-7GHz.npy')[ids]
f0s_design = np.load('LT343_FAB_F0s.npy')[ids]
f0s = freqs[locs]*1e9
f0s_mapped = np.empty(f0s_design.shape)
f0s_mapped[:] = np.nan
df_f = np.empty(kids.shape)
df_f[:] = np.nan
df = np.empty(kids.shape)
df[:] = np.nan
mapped_spacings = np.empty(kids.shape)
mapped_spacings[:] = np.nan
for i in range(M_pxs):
    for j in range(N_pxs):
        design_kid = kid_id_board[i, j]
        design_f0_idx = design_kid
        design_f0 = f0s_design[design_f0_idx]
        mapped_kid = kids[i, j]
        if ~np.isnan(mapped_kid):
            mapped_f0_idx = mapped_kid
            mapped_f0_idx = int(mapped_f0_idx)
            mapped_f0 = f0s[mapped_f0_idx]
            f0s_mapped[design_kid] = mapped_f0
            df[i, j] = (design_f0 - mapped_f0)
            df_f[i, j] = df[i, j] / design_f0
            if design_kid > 1:
                prev_design_kid = design_kid - 1
                prev_i, prev_j = np.argwhere(kid_id_board==prev_design_kid)[0]
                prev_mapped_kid = kids[prev_i, prev_j]
                if ~np.isnan(prev_mapped_kid):
                    prev_mapped_f0_idx = prev_mapped_kid - 1
                    prev_mapped_f0_idx = int(prev_mapped_f0_idx)
                    prev_mapped_f0 = f0s[prev_mapped_f0_idx]
                    mapped_spacings[i, j] = mapped_f0 - prev_mapped_f0
std_df_f = np.std(df_f[~np.isnan(df_f)])

In [ ]:
f0s_design_old = np.load('LEKID_EBEAM_f0s_5-7GHz.npy')[ids]
fig, axes = plt.subplot_mosaic('ab', figsize=(6,3))
ax = axes['a']
ax.scatter(f0s_design_old, f0s_design_old, label='design')
ax.scatter(f0s_design_old, f0s_design, label='design fab')
ax.scatter(f0s_design_old, f0s, label='meas')
ax.legend()
ax = axes['b']
ax.scatter(f0s_design_old, df_f, label='df/f')
ax.legend()

In [ ]:
std_df_f

In [ ]:
np.std((f0s_mapped - f0s_design) / f0s_design)

In [ ]:
fig, ax = plt.subplot_mosaic('ba', figsize=(12, 6), constrained_layout=True)

############################
ax['a'].imshow(kids, origin='lower', cmap='viridis')
for i in range(M_pxs):
    for j in range(N_pxs):
        idx = kids[i, j]
        if ~np.isnan(idx):
            if mask[i, j]:
                color = 'w'
            else:
                color = 'tab:red'
            text = ax['a'].text(j, i, '%d' % idx, ha="center", va="center", color=color, fontsize='x-small') 
# Major ticks
ax['a'].set_xticks(np.arange(0, M_pxs, 1));
ax['a'].set_yticks(np.arange(0, N_pxs, 1));
# Labels for major ticks
ax['a'].set_xticklabels(np.arange(1, M_pxs+1, 1));
ax['a'].set_yticklabels(np.arange(1, N_pxs+1, 1));
# Minor ticks
ax['a'].set_xticks(np.arange(0.5, M_pxs, 1), minor=True);
ax['a'].set_yticks(np.arange(0.5, N_pxs, 1), minor=True);
# labels
ax['a'].set_xlabel('$\it x$ $[px]$')
ax['a'].set_ylabel('$\it y$ $[px]$')
# ax['a'].set_title("Measurement")
# grid
ax['a'].grid(True, which='minor', color='None', linestyle='-', linewidth=0.5)
ax['a'].grid(False, which='major')

############################
im = ax['b'].imshow(kid_id_board, origin='lower', cmap='viridis')
for i in range(M_pxs):
    for j in range(N_pxs):
        idx = kid_id_board[i, j]
        if ~np.isnan(idx):
            text = ax['b'].text(j, i, '%d' % idx, ha="center", va="center", color="w", fontsize='x-small')   
# Major ticks
ax['b'].set_xticks(np.arange(0, M_pxs, 1));
ax['b'].set_yticks(np.arange(0, N_pxs, 1));
# Labels for major ticks
ax['b'].set_xticklabels(np.arange(1, M_pxs+1, 1));
ax['b'].set_yticklabels(np.arange(1, N_pxs+1, 1));
# Minor ticks
ax['b'].set_xticks(np.arange(0.5, M_pxs, 1), minor=True);
ax['b'].set_yticks(np.arange(0.5, N_pxs, 1), minor=True);
# labels
ax['b'].set_xlabel('$\it x$ $[px]$')
ax['b'].set_ylabel('$\it y$ $[px]$')
# ax['b'].set_title("Design")
# grid
ax['b'].grid(True, which='minor', color='None', linestyle='-', linewidth=0.5)
ax['b'].grid(False, which='major')
# cbar = fig.colorbar(im, ax=ax['b'])
# # cbar.ax.invert_yaxis()
# cbar.ax.set_ylabel('$f_0$ $[GHz]$')




In [ ]:

fig, ax = plt.subplot_mosaic('hcc;fcc', figsize=(15, 10), constrained_layout=True)

###################################
# ax['h'].hist(spacings*1e-6, bins=50, alpha=.8, label='before')
ax['h'].hist(mapped_spacings.flatten()*1e-6, bins=40, alpha=1, label='after')
# ax['h'].set_xlim([-100, 100])
# ax['h'].set_ylim([0, 50])
ax['h'].set_xlabel('$d\it{f_0}$ $[MHz]$')
ax['h'].set_ylabel('Counts')

###################################
ax['f'].hist((df_f.flatten()*100), bins=15)
ax['f'].text(8, 50, "$\it\sigma_{\delta f_0/f_0}$ = $%.1f \\times 10^{-3}$" % (std_df_f*1e3))
# ax['f'].set_xlim((7, 11))
ax['f'].set_xlabel('$\it \delta f_0/f_0$ [%]')
ax['f'].set_ylabel('Counts')
# ax['f'].set_ylim([0, 60])
ax['f'].set_title("Frequency scatter, $\it\sigma_{df/f}$ = $%.1f x 10^{-3}$" % (std_df_f*1e3))
# ax['f'].legend()
print('# detected: %d/%d\n# identified: %d/%d\n# swapped: %d/%d ' % (nr_detected, nr_tot, nr_identified, nr_detected, nr_swapped, nr_identified))

###################################
im = ax['c'].imshow(df, origin='lower', cmap='viridis')
# Major ticks
ax['c'].set_xticks(np.arange(0, M_pxs, 1));
ax['c'].set_yticks(np.arange(0, N_pxs, 1));
# Labels for major ticks
ax['c'].set_xticklabels(np.arange(1, M_pxs+1, 1));
ax['c'].set_yticklabels(np.arange(1, N_pxs+1, 1));
# Minor ticks
ax['c'].set_xticks(np.arange(0.5, M_pxs, 1), minor=True);
ax['c'].set_yticks(np.arange(0.5, N_pxs, 1), minor=True);
# labels
ax['c'].set_xlabel('$\it x$ $[px]$')
ax['c'].set_ylabel('$\it y$ $[px]$')
# ax['c'].set_title("$\it\sigma_{df/f}$ = $%.1f x 10^{-3}$" % (std_df_f*1e3))
# grid
ax['c'].grid(True, which='minor', color='None', linestyle='-', linewidth=0.5)
ax['c'].grid(False, which='major')
cbar = fig.colorbar(im, ax=ax['c'])
# cbar.ax.invert_yaxis()
_ = cbar.ax.set_ylabel('$\it \delta f_0/f_0$ [%]')
# print('Unidentified: \n', newhidden_pks)
# plt.savefig('scatter.svg')

In [ ]:
def linear(x, a):
    return a*x

def secorder(x, a, b, c):
    return a*x**2+b*x

def thirdorder(x, a, b, c):
    return a*x**3+b*x**2+c*x


ids = np.arange(400)
nans = np.isnan(f0s_mapped)

line = secorder
flim = np.array([np.amin((f0s_design[~nans], f0s_mapped[~nans]))-.1e9, np.amax((f0s_design[~nans], f0s_mapped[~nans]))+.1e9])
if line:
    if line == linear:
        eq = 'y=ax'
        save = 'fit_linear'
    elif line == secorder:
        eq = 'y=ax^2+bx'
        save = 'fit_secorder'
    elif line == thirdorder:
        eq = 'y=ax^3+bx^2+cx'
        save = 'fit_thirdorder'
    print('line')
    popt, pcov = curve_fit(line, f0s_design[~nans], f0s_mapped[~nans])
    fit = line(f0s_design, *popt)
    xfit = np.linspace(flim[0], flim[1])
    yfit = line(xfit, *popt)
    label = 'y'
else:
    fit = f0s_design
    label = 'f_0^{design}'
    save = 'nofit'

corr_df = (f0s_mapped-fit)
corr_df_f = corr_df / fit
corr_df_map = np.empty(kids.shape)
corr_df_map[:] = np.nan
corr_df_f_map = np.empty(kids.shape)
corr_df_f_map[:] = np.nan
for i, val in enumerate(corr_df):
    x, y = np.argwhere(kid_id_board==i)[0]
    corr_df_map[x, y] = val
    corr_df_f_map[x, y] = corr_df_f[i]

fig, ax = plt.subplot_mosaic('abc', figsize=(16, 5), constrained_layout=True)
ax['a'].scatter(f0s_design*1e-9, f0s_design*1e-9, label='$\it f_0^{design}$')
ax['a'].scatter(f0s_design*1e-9, f0s_mapped*1e-9, label='$\it f_0^{meas}$')
if line:
    ax['a'].plot(xfit*1e-9, yfit*1e-9, c='k', label='fit: $%s$' % eq, ls='--', lw=0.5)
ax['a'].set_xlabel('$\it f_0^{design}$ $[GHz]$')
ax['a'].set_ylabel('$\it f_0$ $[GHz]$')
ax['a'].set_xlim(flim*1e-9)
ax['a'].set_ylim(flim*1e-9)
ax['a'].legend(loc='upper left')
ax['b'].scatter(f0s_design*1e-9, corr_df_f, label='residual, $f_0-y_2$', c=corr_df_f)
ax['b'].set_ylabel('$\it (f_0^{meas}-%s)/%s$' % (label, label))
ax['b'].set_xlabel('$\it f_0^{design}$ $[GHz]$')
im = ax['c'].imshow(corr_df_f_map, origin='lower', cmap='viridis')
# Major ticks
ax['c'].set_xticks(np.arange(0, M_pxs, 1));
ax['c'].set_yticks(np.arange(0, N_pxs, 1));
# Labels for major ticks
ax['c'].set_xticklabels(np.arange(1, M_pxs+1, 1));
ax['c'].set_yticklabels(np.arange(1, N_pxs+1, 1));
# labels
ax['c'].set_xlabel('$\it x$ $[px]$')
ax['c'].set_ylabel('$\it y$ $[px]$')
cbar = fig.colorbar(im, ax=ax['c'])
# cbar.ax.invert_yaxis()
_ = cbar.ax.set_ylabel('$\it (f_0^{meas}-%s)/%s$' % (label, label))
# ax['c'].set_ylim([-100, 50])
# plt.savefig('LT343chip9_scatter_'+save)

In [ ]:
ctr = corr_df_f_map[1:-1, :-1]
np.std(ctr[~np.isnan(ctr)])

In [ ]:
def linear(x, a, b):
    return a*x+b

ctr_ids = np.flip(kid_id_board, 0)[1:-1, :-1]
ctr_ids = ctr_ids[~np.isnan(ctr_ids)]
ctr_f0s = f0s_design[ctr_ids]
ctr_corr_df_f = corr_df_f[ctr_ids]
fig, ax = plt.subplots()
ax.scatter(ctr_f0s, ctr_corr_df_f)
popt, pcov = curve_fit(linear, ctr_f0s, ctr_corr_df_f)
ax.plot(ctr_f0s, linear(ctr_f0s, *popt))
print(np.std(ctr_corr_df_f))
print(np.std(corr_df_f))
